In [9]:
from datetime import datetime
import time

program_start_evening = datetime.strptime("16:29:57", "%H:%M:%S").time()
program_stop_evening = datetime.strptime("17:30:01", "%H:%M:%S").time()
program_start_morning = datetime.strptime("09:10:01", "%H:%M:%S").time()
program_stop_morning = datetime.strptime("09:15:15", "%H:%M:%S").time()
current_time = datetime.now().time()
        # Run this program twice a day (morning and evening) on weekdays (Mon-Fri)
(program_start_morning < current_time < program_stop_morning or program_start_evening < current_time < program_stop_evening)


True

In [4]:
11//3

3

FlatTrade Login

In [24]:
# pip install NorenRestApiPy-0.0.16-py2.py3-none-any.whl

In [3]:
import time
import pandas as pd
# Set display option to show all columns 
pd.set_option('display.max_columns', None)

import pytz
import pyotp
import requests
import json
from urllib.parse import parse_qs,urlparse
import traceback
import hashlib
import login as l 

APIKEY= l.Api_Key
secretKey = l.Api_Secret_Key
totp_key= l.Totp
password =  l.Client_Pass
userid = l.Client_ID
headerJson =  {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36", "Referer":"https://auth.flattrade.in/"}

otp = pyotp.TOTP(totp_key).now()
print("Otp is: ",otp)


max_retries = 2
retry_delay = 5  # seconds
retries = 0

while retries < max_retries:
    try:
        ## Generate Token
        ses = requests.Session()
        sesUrl = 'https://authapi.flattrade.in/auth/session'
        passwordEncrpted = hashlib.sha256(password.encode()).hexdigest()

        res_pin = ses.post(sesUrl, headers=headerJson)
        sid = res_pin.text
        print(f'sid {sid}')

        url2 = 'https://authapi.flattrade.in/ftauth'
        payload = {
            "UserName": userid,
            "Password": passwordEncrpted,
            "PAN_DOB": pyotp.TOTP(totp_key).now(),
            "App": "",
            "ClientID": "",
            "Key": "",
            "APIKey": APIKEY,
            "Sid": sid,
            "Override": "Y",
            "Source": "AUTHPAGE"
        }
        res2 = ses.post(url2, json=payload)
        reqcodeRes = res2.json()
        print(reqcodeRes)

        parsed = urlparse(reqcodeRes['RedirectURL'])  
        reqCode = parse_qs(parsed.query)['code'][0]
        api_secret = APIKEY + reqCode + secretKey
        api_secret = hashlib.sha256(api_secret.encode()).hexdigest()

        payload = {"api_key": APIKEY, "request_code": reqCode, "api_secret": api_secret}
        url3 = 'https://authapi.flattrade.in/trade/apitoken'
        res3 = ses.post(url3, json=payload)
        print(res3.json())

        token = res3.json()['token']
        
        # Create Broker Object
        from NorenRestApiPy.NorenApi import NorenApi
        
        class FlatTradeApiPy(NorenApi):
            def __init__(self):
                NorenApi.__init__(self, host='https://piconnect.flattrade.in/PiConnectTP/', websocket='wss://piconnect.flattrade.in/PiConnectWSTp/', eodhost='https://web.flattrade.in/chartApi/getdata/')
        
        api = FlatTradeApiPy()
        ret = api.set_session(userid=userid, password=password, usertoken=token)

        print("Login successful.")
        break  # Exit loop on success

    except Exception as e:
        print(f"Login Failed: {e}")
        retries += 1
        if retries < max_retries:
            print(f"Retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
        else:
            print("Max retries reached. Login failed.")


Otp is:  293872
sid e9343810b53aad8f513f0da19a7f846669b7d96a38487b844b403f4fe504d134
{'RedirectURL': 'https://www.google.com/?code=181512182ae39d98.1ae351c0bb1f24b54bcefa5f21830083a8b682163fea4f6d3571c75328568ace&client=FZ04385', 'emsg': ''}
{'token': '7dd0183330df0fa9d880f08ca952002aabf1c253d6cd7c7f48724ae17b5a0509', 'client': 'FZ04385', 'stat': 'Ok', 'emsg': ''}
Login successful.


In [ ]:
api.modify_order()

In [27]:
# try:
#     ## Generate Token
#     ses = requests.Session()
#     sesUrl = 'https://authapi.flattrade.in/auth/session'
#     passwordEncrpted =  hashlib.sha256(password.encode()).hexdigest()
#     ses = requests.Session()

#     res_pin = ses.post(sesUrl,headers=headerJson)
#     sid = res_pin.text
#     print(f'sid {sid}')
#     url2 = 'https://authapi.flattrade.in/ftauth'
#     payload = {"UserName":userid,"Password":passwordEncrpted,"PAN_DOB":pyotp.TOTP(totp_key).now(),"App":"","ClientID":"","Key":"","APIKey":APIKEY,"Sid":sid,
#             "Override":"Y","Source":"AUTHPAGE"}
#     res2 = ses.post(url2, json=payload)
#     reqcodeRes = res2.json()
#     print(reqcodeRes)
#     parsed = urlparse(reqcodeRes['RedirectURL'])  
#     reqCode = parse_qs(parsed.query)['code'][0]
#     api_secret =APIKEY+ reqCode + secretKey 
#     api_secret =  hashlib.sha256(api_secret.encode()).hexdigest()
#     payload = {"api_key":APIKEY, "request_code":reqCode, "api_secret":api_secret}
#     url3 = 'https://authapi.flattrade.in/trade/apitoken'  
#     res3 = ses.post(url3, json=payload)
#     print(res3.json())
#     token = res3.json()['token']
#     # token

#     ## Create Broker Object.
#     from NorenRestApiPy.NorenApi import  NorenApi

#     class FlatTradeApiPy(NorenApi):
#         def __init__(self):
#             NorenApi.__init__(self, host='https://piconnect.flattrade.in/PiConnectTP/', websocket='wss://piconnect.flattrade.in/PiConnectWSTp/', eodhost='https://web.flattrade.in/chartApi/getdata/')
            
#     api = FlatTradeApiPy()  
#     ret = api.set_session(userid= userid, password = password, usertoken= token)

# except Exception as e:
#     print(f"Login Failed : {e}\n Retrying to login")
#     try again to login and repaet till login successful




In [28]:
# def fetch_ltp(symbol, retries=3):
#     while retries > 0:
#         try:
#             # Fetch token for the given symbol
#             token_response = api.searchscrip(exchange='NSE', searchtext=symbol)
            
#             if token_response.get('stat') == 'Ok' and isinstance(token_response.get('values'), list):
#                 token = token_response['values'][0]['token']
                
#                 # Fetch LTP using the obtained token
#                 ltp_response = api.get_quotes(exchange='NSE', token=token)
                
#                 if ltp_response.get('stat') == 'Ok' and 'lp' in ltp_response:
#                     ltp = ltp_response['lp']
#                     return ltp
#                 else:
#                     print("Error fetching LTP:", ltp_response.get('emsg', 'Unknown error'))
#             else:
#                 print("Error fetching token:", token_response.get('emsg', 'Invalid response or no values found.'))
                
#         except Exception as e:
#             print(f"Exception occurred: {e}")
        
#         # Reduce retry count and attempt again if retries remain
#         retries -= 1
#         if retries > 0:
#             print(f"Retrying... ({retries} attempts left)")
#         else:
#             print("Max retries reached. Could not fetch LTP.")
    
#     return None


FlatTrade Place Order Function

In [7]:
from datetime import datetime, timedelta
import time
import threading
import database as db 
import mail 

fund_each_stock = 650 *5  ##change the value accordingly

MAX_RETRIES = 2  # Set the maximum number of retries

def fetch_ltp(symbol):
    try:
        token_response = api.searchscrip(exchange='NSE', searchtext=symbol)
        if token_response.get('stat') == 'Ok' and isinstance(token_response.get('values'), list):
            token = token_response['values'][0]['token']
            ltp_response = api.get_quotes(exchange='NSE', token=token)
            return float(ltp_response['lp'])
        else:
            raise ValueError("Failed to fetch LTP: Invalid response")
    except Exception as e:
        print(f"Error fetching LTP for {symbol}: {str(e)}")
        return None


import pandas as pd
from datetime import datetime

def is_mis(symbol):
    df = pd.read_excel("Margins.xlsx", skiprows=1)
    df = df.drop(columns=['MIS/CO/BO.1', 'MTF.1'])
    df = df.dropna()  # Remove all records where rows are NA for MIS or NTF or any
    symbol = symbol.split('-')[0]  # Remove "-EQ" from the symbol
    
    ## Make sure it is morning time, because planning MIS for morning order only
    current_time = datetime.now()
    cutoff_time = current_time.replace(hour=9, minute=16, second=0, microsecond=0)
    
    if df['SYMBOL'].str.lower().eq(symbol.lower()).any() and current_time < cutoff_time:
        return True
    else:
        return False




def exit_order(symbol,MIS_or_MTF, Qty):
    ltp_price = fetch_ltp(symbol)
    
    if MIS_or_MTF == 'Mis' or MIS_or_MTF == 'MIS' or MIS_or_MTF == 'I' or MIS_or_MTF == 'i':
        MIS_or_MTF = 'I'
    else :
        MIS_or_MTF = 'F'
    
    if ltp_price is None:
        print(f"Failed to fetch LTP for {symbol}. Skipping order.")
        return None

    try:
        ret  = api.place_order(
            act_id        =  l.Client_ID,   
            buy_or_sell   =  'S',
            product_type  =  MIS_or_MTF,  ## We are going to use  MTF-F or MIS-I
            exchange      =  'NSE',
            tradingsymbol =  symbol,
            quantity      =  Qty, 
            discloseqty   =  0,
            price_type    =  'MKT',
            price         =   0,
            trigger_price =  None,
            retention     =  'DAY',
            remarks       =  f'Placed a Sell order to exit {symbol} position '
        )
        print(f"Order placed successfully for {symbol}:", ret)
        return ret
    except Exception as e:
        print(f"Failed to place order for {symbol}: {str(e)}")
        return None


def place_order(symbol,Qty ):

    # ## Deciding Product type
    # if MIS_or_MTF == 'Mis' or MIS_or_MTF == 'MIS' or MIS_or_MTF == 'I' or MIS_or_MTF == 'i':
    #     MIS_or_MTF = 'I'
    # else :
    #     MIS_or_MTF = 'F'
    
    
    try:
        ret = api.place_order(
            act_id=l.Client_ID,   
            buy_or_sell='B',
            product_type= "I",  
            exchange='NSE',
            tradingsymbol=symbol,
            quantity=Qty,
            discloseqty=0,
            price_type='MKT',
            price=0,
            trigger_price=None,
            retention='DAY',
            remarks='Order placed via script'
        )
        print(f"Order placed successfully for {symbol}:", ret)
        return ret
    except Exception as e:
        print(f"Failed to place order for {symbol}: {str(e)}")
        return None



def live_monitor(symbol):
    while True:  # In Case of threading
        try:
            ltp = fetch_ltp(symbol)
            ltp = float(ltp)
        except:
            print(f"Failed to fetch LTP of {symbol}")
            time.sleep(2.5)
            continue  # Skip the rest of the loop and try again

        qty = db.get_qty(symbol, ltp)

        if int(qty) > 0:
            if is_mis():  # Exit MIS order
                order = exit_order(symbol, "I", qty)
                if order:
                    print(f"Order Exit Successfully {symbol}, Qty: {qty}")
                    db.db_exit(symbol, ltp)  # Make exit entry into db
                    mail.mail_with_text(f"Order Completed {symbol}", f"A Order has been completed and Exit made\n\n Symbol: {symbol}  Exit_price: {ltp}")
                    break  # Exit the main while loop after successful order exit
            else:  # Exit MTF order
                order = exit_order(symbol, "F", qty)
                if order:
                    print(f"Order Exit Successfully {symbol}, Qty: {qty}")
                    db.db_exit(symbol, ltp)  # Make exit entry into db
                    mail.mail_with_text(f"Order Completed {symbol}", f"A Order has been completed and Exit made\n\n Symbol: {symbol}  Exit_price: {ltp}")
                    break  # Exit the main loop after successful order exit



# Corrected stocks dictionary
stocks = {
    # "EDELWEISS-EQ": 1,
    # "NH-EQ": 2,
    # "ABCAPITAL-EQ": 3,  ## yes
    # "TEJASNET-EQ": 4,
    # "GRAVITA-EQ": 5,
    # "KCP-EQ": 6,
    # "NATCOPHARM-EQ": 7,
    # "SCHNEIDER-EQ": 8,
    # "CESC-EQ": 9,   ## yes
    # "GESHIP-EQ": 10,
    # "LATENTVIEW-EQ": 11,
    # "MAPMYINDIA-EQ": 12,
    # "HAL-EQ": 13,   ##yes
    # "HINDCOPPER-EQ": 14,  ##yes
    # "TITAN-EQ": 15,  ##yes
    # "CYIENT-EQ": 16,  ##yes
    # "MOLDTECH-EQ": 17,
    # "INGERRAND-EQ": 18,
    # "MAZDOCK-EQ": 19,
    # "JMFINANCIL-EQ": 20,
    "SONACOMS-EQ": 21,  ##yes
    # "CDSL-EQ": 22,  ##yes
    # "ACE-EQ": 23,
    # "TATAELXSI-EQ": 24,  ##yes
    # "ROHLTD-EQ": 25,
    # "CANFINHOME-EQ": 26,  ##yes
    # "TRITURBINE-EQ" : 27,
    # "TBOTEK-EQ": 28,
    # "VPRPL-EQ" : 29,
}

start_time_evening = datetime.strptime("13:29:57", "%H:%M:%S").time()
stop_time_evening = datetime.strptime("15:30:01", "%H:%M:%S").time()
start_time_morning = datetime.strptime("09:15:01", "%H:%M:%S").time()
stop_time_morning = datetime.strptime("09:15:15", "%H:%M:%S").time()


def place_order_in_thread(symbol):
    
    ltp = fetch_ltp(symbol)
    if ltp is not None:
        qntty = int(fund_each_stock // ltp)
        
    if place_order(symbol,qntty):
        print(f"Order placed for {symbol} at {datetime.now()}")
        db.db_entry(symbol, ltp, qntty)
    else:
        print(f"Order placement failed for {symbol}. Retrying...")



while True:
    current_time = datetime.now().time()

    if (start_time_morning <= current_time <= stop_time_morning) or (start_time_evening <= current_time <= stop_time_evening):
        time_frame = "Morning" if start_time_morning <= current_time <= stop_time_morning else "Evening"
        print(f"\n================== {time_frame} Time =========================")

        threads = []
        for symbol in stocks.keys():
            t = threading.Thread(target=place_order_in_thread, args=(symbol,))
            threads.append(t)
            t.start()
        
        for t in threads:
            t.join()
        
        break  # Exit main loop after completing one cycle for all stocks
    else:
        current_datetime = datetime.combine(datetime.today(), current_time)
        next_morning_start = datetime.combine(datetime.today(), start_time_morning)
        next_evening_start = datetime.combine(datetime.today(), start_time_evening)

        if current_time > stop_time_evening:
            # Calculate remaining time until next morning start
            next_morning_start += timedelta(days=1)
            remaining_time = next_morning_start - current_datetime
        elif current_time > stop_time_morning:
            # Calculate remaining time until next evening start
            remaining_time = next_evening_start - current_datetime
        else:
            # Calculate remaining time until next closest start time
            remaining_time = min(next_morning_start, next_evening_start) - current_datetime

        print(f"\rCurrent Time: {current_time}  || Remaining time: {remaining_time}", end="", flush=True)

    time.sleep(1)  # Wait for 1 second before checking the time again



================== Evening Time =========================
Order placed successfully for SONACOMS-EQ: {'request_time': '14:38:15 27-12-2024', 'stat': 'Ok', 'norenordno': '24122700634906'}
Order placed for SONACOMS-EQ at 2024-12-27 14:38:15.354146
